# Obtain data from wiki page

# Toronto Neighborhoods

# . import

In [19]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim


### . will use 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
### . check for status code and headers

In [20]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(url)
print(url)
print(result.status_code)
print(result.headers)

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
200
{'Date': 'Wed, 06 May 2020 11:40:19 GMT', 'Server': 'mw1264.eqiad.wmnet', 'X-Content-Type-Options': 'nosniff', 'P3p': 'CP="See https://en.wikipedia.org/wiki/Special:CentralAutoLogin/P3P for more info."', 'Content-Language': 'en', 'Vary': 'Accept-Encoding,Cookie,Authorization', 'Last-Modified': 'Wed, 06 May 2020 10:53:36 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Content-Encoding': 'gzip', 'Age': '18430', 'X-Cache': 'cp2041 hit, cp2041 hit/2', 'X-Cache-Status': 'hit-front', 'Server-Timing': 'cache;desc="hit-front"', 'Strict-Transport-Security': 'max-age=106384710; includeSubDomains; preload', 'Set-Cookie': 'WMF-Last-Access=06-May-2020;Path=/;HttpOnly;secure;Expires=Sun, 07 Jun 2020 12:00:00 GMT, WMF-Last-Access-Global=06-May-2020;Path=/;Domain=.wikipedia.org;HttpOnly;secure;Expires=Sun, 07 Jun 2020 12:00:00 GMT, GeoIP=US:::37.75:-97.82:v4; Path=/; secure; Domain=.wikipedia.org', 'X-Client-IP': '169.62.147.26', '

### . will get data + clean it

In [21]:
soup = BeautifulSoup(result.content, 'html.parser')
table = soup.find('table')
trs = table.find_all('tr')
rows = []
for tr in trs:
    i = tr.find_all('td')
    if i:
        rows.append(i)
        
lst = []
for row in rows:
    postalcode = row[0].text.rstrip()
    borough = row[1].text.rstrip()
    neighborhood = row[2].text.rstrip()
    if borough != 'Not assigned':
        if neighborhood == 'Not assigned':
            neighborhood = borough
        lst.append([postalcode, borough, neighborhood])

# lst

### . convert into a dataframe

In [22]:
cols = ['PostalCode', 'Borough', 'Neighborhood']
df = pd.DataFrame(lst, columns=cols)
print(df.shape)
# df[df.duplicated(['PostalCode'], keep=False)] - this would have shown the duplicate PostalCodes

(103, 3)


In [24]:
df = df.groupby('PostalCode').agg(
    {
        'Borough':'first', 
        'Neighborhood': ', '.join,}
    ).reset_index()

# could have done:
# df = df.groupby(['Postcode', 'Borough'], as_index=False).agg(lambda neighborhoods: ', '.join(neighborhoods))

In [25]:
df.loc[df['PostalCode'] == 'M5A']

,PostalCode,Borough,Neighborhood
53,M5A,Downtown Toronto,Regent Park / Harbourfront


In [28]:
df.head(12)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park
7,M1L,Scarborough,Golden Mile / Clairlea / Oakridge
8,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West
9,M1N,Scarborough,Birch Cliff / Cliffside West


In [29]:
df.shape

(103, 3)